In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import torch
import torch.nn as nn
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

In [3]:
pyro.set_rng_seed(0)

In [4]:
df = pd.read_excel('pack_benchmark_spec_v1.055.xlsm','Cells',header=[0,1],skiprows=1,nrows=1102)

## Data analysis

In [9]:
feature_set_0 = [('Capacity','Ah (nom)'),('Voltage','Nom'),
                 ('Mass','kg'),('Format','Unnamed: 14_level_1'),
                 ('Chemistry','Family'),('Chemistry','Cathode'),('Chemistry','Anode'),
                 ('Discharge','Ampsmax'),('Discharge','Ampscont'),
                 ('Charge','Ampsmax'),('Charge','Ampscont'),
                 ('Cycle Ageing','to 70% SoH'),('Cycle Ageing','to 80% SoH')]

In [10]:
df.loc[:,feature_set_0].describe()

Capacity      Voltage        Mass     Discharge               \
          Ah (nom)          Nom          kg       Ampsmax     Ampscont   
count  1099.000000  1049.000000  986.000000    261.000000   730.000000   
mean     53.910874     3.467774    1.225870    298.912859   123.278508   
std      97.407426     0.509421    2.674496    719.616163   252.415326   
min       0.026000     1.200000    0.004200      2.500000     0.020000   
25%       3.400000     3.200000    0.061475     20.000000    10.087500   
50%      10.000000     3.600000    0.269000    120.000000    33.500000   
75%      62.750000     3.700000    1.420000    380.000000   150.000000   
max    1175.000000    12.000000   38.000000  10000.000000  3000.000000   

            Charge              Cycle Ageing               
           Ampsmax    Ampscont    to 70% SoH   to 80% SoH  
count   414.000000  657.000000     84.000000    183.00000  
mean     99.097080   37.085752   3179.166667   3140.73224  
std     264.029472   74.246094   3265.594729   4662.17618  
min       0.825000    0.008000    200.000000    250.00000  
25%       5.000000    2.000000    600.000000   1000.00000  
50%      20.000000    4.400000   2000.000000   2000.00000  
75%     110.000000   30.000000   5250.000000   3500.00000  
max    3000.000000  500.000000  13000.000000  35000.00000

In [11]:
df.loc[:,feature_set_0].describe(include=['object','category'])

Format Chemistry                              
       Unnamed: 14_level_1    Family                 Cathode Anode
count                 1100       797                     436   118
unique                   6        23                      17    16
top            Cylindrical       LFP  Lithium Iron Phosphate    Gr
freq                   468       368                     211    48

In [12]:
df.loc[:,'Format'].value_counts()

Unnamed: 14_level_1
Cylindrical            468
Prismatic              335
Pouch                  266
Blade                   21
cylindrical              8
Cylindrical              2
Name: count, dtype: int64

In [13]:
df.loc[:,('Chemistry','Family')].value_counts()

(Chemistry, Family)
LFP                    368
NMC                    197
NCA                     56
NMC, NCA, NMC/NCA       49
LCO                     40
LTO                     19
NaB                     17
NCM/NCA                 13
LMO                      7
Solid State              5
PbA                      4
NiCd                     4
NMCA                     3
NiMh                     3
LiS                      2
LMFP                     2
NMX                      2
BMLMP                    1
NMC 811                  1
Spinel                   1
Silicon carbon cell      1
LNCAO                    1
Li-S                     1
Name: count, dtype: int64

In [14]:
df.loc[:,('Chemistry','Cathode')].value_counts()

(Chemistry, Cathode)
Lithium Iron Phosphate    211
Nickel rich               151
Lithium Cobalt Oxide       33
NMC811                     12
NMC622                      7
Li metal/Liquid             5
Lithium Titanate            4
Gr+Si                       3
NaMO2                       2
NMC523                      1
NMC                         1
712                         1
NMC712                      1
NMC111+NCA                  1
High Nickel NCM+ Gr         1
High Nickel + SiC/SiOx      1
NMC955                      1
Name: count, dtype: int64

In [15]:
df.loc[:,('Chemistry','Anode')].value_counts()

(Chemistry, Anode)
Gr                  48
Graphite            26
Si                   9
Li                   8
Aluminium            8
Gr/SiO               5
Gr-Si                4
Li metal             2
LTO                  1
Hard carbon          1
SiC                  1
Graphite+Silicon     1
Soft carbon          1
Gr+LTO               1
Gr+SiO               1
Gr+Si                1
Name: count, dtype: int64

## ML model
We will use the VAE-based model from [this paper](https://www.sciencedirect.com/science/article/pii/S0031320320303046#bib0045). The basic idea is to use a neural network model to 'reconstruct' the data. This is done by forcing it through a bottleneck in the NN, the _latent space_.